In [ ]:
import gymnasium as gym
from highway_env.envs.parking_env import ParkingEnv
import numpy as np
from highway_env.vehicle.objects import Landmark,Obstacle
from highway_env.vehicle.kinematics import Vehicle
from highway_env.vehicle.graphics import VehicleGraphics
import random

class ParkingWithObstacles(ParkingEnv):
    def __init__(self, env):
        # Set subclass attributes before initializing the parent class
        self.env = env
        self.open_walls = False # set the wall around the parking lot
        self.num_obstacles = 3  # Set number of obstacles
        self.num_init_vehocles = 3 # Set number of initial vehicles
        self.collision_reward = -50 # set collision reward
        self.success_goal_reward = 100 # set goal reward
        self.further_reward = -10 # not yet

        # observation

        ## type
        ## KinematicsGoal indicates that the observation type is based on kinematics,
        ## meaning the observation will include information about the object's motion state
        
        ## feature
        ##  x: The object's x-coordinate (position)
        ##  y: The object's y-coordinate (position)
        ##  vx: The object's velocity along the x-axis
        ##  vy: The object's velocity along the y-axis
        ##  cos_h: The cosine of the object's heading (used to represent its direction)
        ##  sin_h: The sine of the object's heading (also used to represent its direction)
        
        ## scale
        ## 100, meaning the position values are scaled by a factor of 100.
        ## 5, meaning the velocity values are scaled by a factor of 5.
        ## 1, meaning the direction values are not scaled.

        ## normalize
        ## normalize determines whether the observation values should be normalized. 
        ## If set to True, the observations would be adjusted to fit within a specific range, 
        ## such as [0, 1]

        # action
        
        ## type
        ## DiscreteMetaAction means that actions is not continuous.
        ## 1. LANE_LEFT
        ## 2. IDLE
        ## 3. LANE_RIGHT
        ## 4. FASTER
        ## 5. SLOWER
        ##
        ## examples:
        ## ACTIONS_ALL = {0: 'LANE_LEFT', 1: 'IDLE', 2: 'LANE_RIGHT', 3: 'FASTER', 4: 'SLOWER'}
        
        config = {
            "observation": {
                "type": "KinematicsGoal",
                "features": ['x', 'y', 'vx', 'vy', 'cos_h', 'sin_h'],
                "scales": [100, 100, 5, 5, 1, 1],
                "normalize": True
            },
            "action": {
                "type": "DiscreteMetaAction",
            },
            "screen_height": 600,
            "screen_width": 1200,
            "vehicles_count": self.num_init_vehocles,  # No initial vehicles
            "add_walls": self.open_walls,   # Turn off automatic walls, as we'll add obstacles manually
            "add_obstacles": True,
            "obstacles_count": self.num_obstacles, 
            "centering_position": [0.5, 0.5],
            "scaling": 9,
            "controlled_vehicles": 1,
            "collision_reward": self.collision_reward,
            "success_goal_reward": self.success_goal_reward,
        }

        # Initialize the parent class
        super().__init__(config=config, render_mode=env.render_mode)

        # Set observation space and action space to match the original environment
        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space

    def _create_vehicles(self) -> None:
        """Create some new random vehicles of a given type, and add them on the road."""
        empty_spots = list(self.road.network.lanes_dict().keys())

        # Controlled vehicles
        self.controlled_vehicles = []
        for i in range(self.config["controlled_vehicles"]):
            x0 = (i - self.config["controlled_vehicles"] // 2) * 10
            vehicle = self.action_type.vehicle_class(
                self.road, [x0, 0], 2 * np.pi * self.np_random.uniform(), 0
            )
            vehicle.color = VehicleGraphics.EGO_COLOR
            self.road.vehicles.append(vehicle)
            self.controlled_vehicles.append(vehicle)
            empty_spots.remove(vehicle.lane_index)

        # Goal
        for vehicle in self.controlled_vehicles:
            lane_index = empty_spots[self.np_random.choice(np.arange(len(empty_spots)))]
            lane = self.road.network.get_lane(lane_index)
            vehicle.goal = Landmark(
                self.road, lane.position(lane.length / 2, 0), heading=lane.heading
            )
            self.road.objects.append(vehicle.goal)
            empty_spots.remove(lane_index)

        # The other goals

        # Other vehicles
        for i in range(self.config["vehicles_count"]):
            if not empty_spots:
                continue
            lane_index = empty_spots[self.np_random.choice(np.arange(len(empty_spots)))]
            v = Vehicle.make_on_lane(self.road, lane_index, 4, speed=0)
            self.road.vehicles.append(v)
            empty_spots.remove(lane_index)

        # Walls
        if self.config["add_walls"]:
            width, height = 70, 42
            for y in [-height / 2, height / 2]:
                obstacle = Obstacle(self.road, [0, y])
                obstacle.LENGTH, obstacle.WIDTH = (width, 1)
                obstacle.diagonal = np.sqrt(obstacle.LENGTH**2 + obstacle.WIDTH**2)
                self.road.objects.append(obstacle)
            for x in [-width / 2, width / 2]:
                obstacle = Obstacle(self.road, [x, 0], heading=np.pi / 2)
                obstacle.LENGTH, obstacle.WIDTH = (height, 1)
                obstacle.diagonal = np.sqrt(obstacle.LENGTH**2 + obstacle.WIDTH**2)
                self.road.objects.append(obstacle)

        # Obstacles
        if self.config["add_obstacles"]:
            for i in range(self.config["obstacles_count"]):
                lane_index = empty_spots[self.np_random.choice(np.arange(len(empty_spots)))]
                lane = self.road.network.get_lane(lane_index)
                position = lane.position(lane.length / 2, 0)
                position[1] = round(random.uniform(0, position[1]))
                size = 2  # Set the side length of square obstacles
                # Create a square obstacle in the center of the map
                obstacle = Obstacle(self.road, position)  # Place it at the center of the map
                obstacle.LENGTH = obstacle.WIDTH = size  # Set to square
                obstacle.diagonal = np.sqrt(obstacle.LENGTH**2 + obstacle.WIDTH**2)
                self.road.objects.append(obstacle)
                empty_spots.remove(lane_index)

# Create the original environment and wrap it into an environment with obstacles
env_origin = gym.make("parking-v0", render_mode="human")
env = ParkingWithObstacles(env_origin)
env.define_spaces()

done = False
truncated = False

# Print the number of states and actions
print(f"Observation Space: {env.observation_space}")
# achieved_goal Box(-inf, inf, (6,), float64)
# desired_goal Box(-inf, inf, (6,), float64)
# observation Box(-inf, inf, (6,), float64)

print(f"Action Space: {type(env.action_space)}")
# Box(-1.0, 1.0, (2,), float32)

# while not (done or truncated):
action = env.action_space.sample()  # Randomly sample an action
obs, reward, done, truncated, info = env.step(action)

# Uncomment to view information for each step
print(f"Obs: {obs}, Reward: {reward}, Done: {done}, Truncated: {truncated}")

# print(done)
# print(truncated)
# env.close()

Observation Space: Dict('achieved_goal': Box(-inf, inf, (6,), float64), 'desired_goal': Box(-inf, inf, (6,), float64), 'observation': Box(-inf, inf, (6,), float64))
Action Space: <class 'gymnasium.spaces.discrete.Discrete'>
Obs: OrderedDict([('observation', array([-0.00221972,  0.00365836, -1.09741787,  0.46192444, -0.9216793 ,
        0.38795267])), ('achieved_goal', array([-0.00221972,  0.00365836, -1.09741787,  0.46192444, -0.9216793 ,
        0.38795267])), ('desired_goal', array([-6.000000e-02,  1.400000e-01,  0.000000e+00,  0.000000e+00,
        6.123234e-17,  1.000000e+00]))]), Reward: -0.35966276973870603, Done: True, Truncated: False


In [23]:
env.close()